In [ ]:
!pip install -qU langchain_ollama
!pip install -qU "langchain-chroma>=0.1.2"

In [67]:
%cd C:\Users\natha\OneDrive\Documents\University of Michigan\Capstone\

C:\Users\natha\OneDrive\Documents\University of Michigan\Capstone


In [68]:
from chromadb.utils import embedding_functions
import pandas as pd
import chromadb

default_ef = embedding_functions.DefaultEmbeddingFunction()

In [71]:
# Run if need to create a new dataset
import chromadb
client = chromadb.PersistentClient(path="collection_test")
collection = client.create_collection(name="trial_collection", embedding_function=default_ef)


In [70]:
# Run if need to delete collection
client.delete_collection(name="trial_collection")

In [82]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Take text field and add it to ChromaDB
def text_collection_add(text_pdSeries,collection, section, ticker):
    text = (' '.join(text_pdSeries['text']))

    # Set Text Splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        separators=["\n\n", "\n", ". ", " ", ""],
    )

    # Create Docs
    docs = text_splitter.create_documents([text])

    # Add to ChromaDB
    collection.add(
        documents=[i.page_content for i in docs],
        ids=[ticker + "_" + section + "_" + f"{i}" for i in range(len(docs))],
        metadatas=
        [
            {
                "ticker": ticker,
                "section":section}
            for _ in docs
        ],
    )


In [73]:
collection = client.get_collection(name="trial_collection", embedding_function=default_ef)

In [ ]:
for ticker in ['A','AAPL','ABBV','ABT','ACGL','ACN','ADBE','ADI','ADM','ADP','ADSK']:
    print(ticker)
    doc = pd.read_csv('DOCS/'+ticker+'_2024.csv')
    for s in doc['section'].unique():
        print(s)
        text_collection_add(doc[doc['section'] == s],collection, s, ticker)

A
PROPERTIES
LEGAL_PROCEEDINGS
MANAGEMENT_DISCUSSION


In [77]:
collection.peek(5)

{'ids': [],
 'embeddings': array([], dtype=float64),
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.embeddings: 'embeddings'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [51]:
collection.query(query_texts=["What are the main properties?"], n_results=2)

{'ids': [['A_PROPERTIES_0', 'A_LEGAL_PROCEEDINGS_0']],
 'embeddings': None,
 'documents': [['section element_type text', 'section element_type text']],
 'uris': None,
 'data': None,
 'metadatas': [[{'section': 'PROPERTIES', 'ticker': 'A'},
   {'section': 'LEGAL_PROCEEDINGS', 'ticker': 'A'}]],
 'distances': [[1.768758593799916, 1.768758593799916]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}